**INTRO**

Questo notebook è da eseguire idealmente in un environment Google Colab.
Se si dispone di GPU NVIDIA locale, questo notebook può rappresentare uno scheletro delle operazioni essenziali ma deve essere modificato.

Per far funzionare questo notebook caricare all'interno di Google Drive la "cartella-progetto" ottenuta con il notebook adibito al "preprocessing" (senza GPU). Nota che prima di inziare, se necessario, bisogna modificare il file `pose_cfg.yaml`.

In [ ]:
# Installs a CUDA version compatible with tensorflow on COLAB
!apt update && apt install cuda-11-8

# Install the latest DeepLabCut version:
!pip install "deeplabcut[tf, gui]"

In [ ]:
import deeplabcut
import os
from google.colab import drive

# Mount the drive
drive.mount('/content/drive')
# Set the project path
project_path = "/content/drive/MyDrive/IZSVE/Prova_Video_Rachele-Tommy-2024-03-19"
# Set the config path
config_path = os.path.join(project_path, "config.yaml")

# Training the network
**maDeepLabCut CRITICAL POINT**: For multi-animal projects we are using not only different and new output layers, but also new data augmentation, optimization, learning rates, and batch training defaults. Thus, please use a lower save_iters and maxiters. I.e. we suggest **saving every 10K-15K iterations**, and only **training until 50K-100K iterations**. We recommend you look closely at the loss to not overfit on your data. The bonus, training time is much less!!!

`config` : string
    Full path of the config.yaml file as a string.

`shuffle`: int, optional
    Integer value specifying the shuffle index to select for training. Default is set to 1

`trainingsetindex`: int, optional
    Integer specifying which TrainingsetFraction to use. By default the first (note that TrainingFraction is a list in config.yaml).

`gputouse`: int, optional. Natural number indicating the number of your GPU (see number in nvidia-smi). If you do not have a GPU, put None. See: https://nvidia.custhelp.com/app/answers/detail/a_id/3751/~/useful-nvidia-smi-queries

`max_snapshots_to_keep`: int, or None. Sets how many snapshots are kept, i.e. states of the trained network. For every saving iteration a snapshot is stored, however, only the last max_snapshots_to_keep many are kept! If you change this to None, then all are kept. See: https://github.com/DeepLabCut/DeepLabCut/issues/8#issuecomment-387404835

`autotune`: property of TensorFlow, somehow faster if 'false' (as Eldar found out, see https://github.com/tensorflow/tensorflow/issues/13317). Default: False

`displayiters`: this variable is actually set in pose_config.yaml. However, you can overwrite it with this hack. Don't use this regularly, just if you are too lazy to dig out the pose_config.yaml file for the corresponding project. If None, the value from there is used, otherwise it is overwritten! Default: None

`saveiters`: this variable is actually set in pose_config.yaml. However, you can overwrite it with this hack. Don't use this regularly, just if you are too lazy to dig out the pose_config.yaml file for the corresponding project. If None, the value from there is used, otherwise it is overwritten! Default: None

`maxiters`: This sets how many iterations to train. This variable is set in pose_config.yaml. However, you can overwrite it with this. If None, the value from there is used, otherwise it is overwritten! Default: None

In [ ]:
deeplabcut.train_network(config_path, allow_growth=True)

# Network Evaluation

In [ ]:
deeplabcut.evaluate_network(config_path, plotting=True)

Usando questa funzione si creano:
+ Un file `.csv` contenente il Mean Average Euclidean error (MAE).
+ I risultati plottati sulle immagini di training/test e contenuti nel folder `evaluation-result`.

In [ ]:
# plot the scoremaps, locref layers, and PAFs to assess performance
deeplabcut.extract_save_all_maps(config_path, Indices=[0, 5])

I risultati di questa funzione sono salvati all'interno di `\evaluation-results\iteration-0\proj_name\maps`

# Prima di continuare
Prima di analizzare degli interi video, dobbiamo controllare che le metriche al punto precedente siano "buone abbastanza". Se la precisione delle predictions è soddisfacente possiamo passare al prossimo punto.

In [ ]:
# Utility per creare la lista di video

def select_videos():
    print("Seleziona una cartella contenente i video.")

    # Funzione per ottenere la cartella selezionata dall'utente
    def select_folder():
        root = Tk()
        root.withdraw()  # Nasconde la finestra principale
        folder_selected = filedialog.askdirectory(title="Seleziona una cartella")
        return folder_selected
    
    # Ottieni la cartella selezionata dall'utente
    folder_path = select_folder()
    
    # Inizializza una lista per contenere i percorsi dei video
    video_list = []
    
    # Scansiona la cartella selezionata e aggiungi i video alla lista
    for file_name in os.listdir(folder_path):
        if file_name.endswith(('.mp4', '.avi', '.mov', '.mkv')):
            video_list = os.path.join(folder_path, file_name)
            video_list.append(video_list)
    
    # Ritorna la lista dei percorsi dei video
    return video_list

In [ ]:
video_list = select_videos()
auto_track = True # Set False se non si vuole fare in unico passaggio chaining pose estimation, tracking e sticthing
scorername = deeplabcut.analyze_videos(config_path, video_list, videotype='.mp4')
deeplabcut.create_video_with_all_detections(config_path, video_list, videotype='.mp4')

Un ulteriore check è quello di analizzare se la NN è riuscita a imparare la correlazione tra diverse bodyparts dello stesso animale. Per fare questo si usa la funzione `deeplabcut.utils.plot_edge_affinity_distributions(eval_pickle_file, include_bodyparts="all", output_name="", figsize=(10, 7))`.
Se le distribuzioni sono facilmente separabili allora il modello ha avuto successo nell'imparare relazioni tra diverse bodyparts.

**N.B.**: Questo tipo di valutazione è particolarmente importante nel caso di progetti con più animali, visto che le occlusioni potrebbero altrimenti far calare la precisione del modello.

In [ ]:
# Path to a full pickle file from the evaluation-results folder
eval_pickles = [r'C:\Users\bergamascot\Documents\Progetti\DEEPLABCUT PROJECTS\Prova_Video_Rachele-Tommy-2024-03-19-already_trained\Prova_Video_Rachele-Tommy-2024-03-19\evaluation-results\iteration-0\Prova_Video_RacheleMar19-trainset95shuffle1\DLC_resnet50_Prova_Video_RacheleMar19shuffle1_45000-snapshot-45000_map.pickle']
deeplabcut.utils.plot_edge_affinity_distributions(eval_pickles)